# Similarity
Now that we can produce colour palettes for images, we can move on to the second task - asserting similarity between images based on the colours present within them.  
For the task of producing notebooks, researching previous literature alone was sufficient to produce satisfactory results. However, because of the difference between our domain and more typical image datasets, I think it's worth implementing a few different approaches in code to determine which is most appropriate/effective. We'll start with histograms.

### Image histograms
As we saw in the last notebook, images are composed of pixels, and the colour of those pixels can be represented by three numbers (alternatively seen as coordinates in colour space). We can stack the pixels in an image to produce a table of three columns or channels (traditionally R, G, and B, where each channel can take on a value between 0 and 255 denoting its intensity). 
A common technique to assert the 'makeup' of an image is to produce a _histogram_ of these channels, counting the number of times that each intensity appears in each channel. Our goal in this notebook is to determine an effective way of comparing these histograms

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (20, 15)

import os
import itertools
import numpy as np
import pandas as pd
from PIL import Image
from scipy.spatial.distance import cosine
from umap import UMAP

from tqdm import tqdm_notebook as tqdm

Lets randomly load in a few images and get a visual sense of the distribution of colours/tones/styles across our chosen dataset.

In [ ]:
n_images = 1000
path_to_images = '../data/small_images/'

image_ids = np.random.choice(os.listdir(path_to_images), n_images, replace=False)
images = [Image.open(path_to_images + image_id) for image_id in tqdm(image_ids)]
images = [Image.fromarray(np.stack((image,)*3, -1))
          if len(np.array(image).shape) != 3 else image
          for image in images]

In [ ]:
resolution = 75
size = int(n_images ** 0.5)

height = int(resolution * size)
width = int(resolution * size)

big_image = np.empty((height, width, 3)).astype(np.uint8)
grid = np.array(list(itertools.product(range(size), range(size))))
sq_images = [image.resize((resolution, resolution)) for image in images]

for pos, image in zip(grid, sq_images):
    block_t, block_l = pos * resolution
    block_b, block_r = (pos + 1) * resolution
    
    big_image[block_t : block_b, block_l : block_r] = np.array(image)

Image.fromarray(big_image)

# Histograms

In [ ]:
image = images[np.random.choice(len(images))]
hist = image.histogram()

image

this is what our raw histogram data looks like

In [ ]:
pd.Series(hist).plot();

and here it is split out into its three colour channels in the 

In [ ]:
r, g, b = np.split(np.array(hist), 3)

pd.DataFrame({'r': r, 'g': g, 'b': b}).plot(color=['#333399', '#339933', '#993333']);

# A naive start
Lets blunder straight in with the simplest approach possible. We'll produce a historgram for each image as one long 756-dimensional vector and then brute force the cosine similarity of each image with every other image.

In [ ]:
histograms = [image.histogram() for image in images]

In [ ]:
naive_similarity = pd.DataFrame(data=[[cosine(h_1, h_2)
                                       for h_1 in histograms] 
                                      for h_2 in tqdm(histograms)],
                                index=image_ids,
                                columns=image_ids)

we can display the similarity matrix as a heatmap, as follows:

In [ ]:
sns.heatmap(naive_similarity);

In [ ]:
image_dict = dict(zip(image_ids, images))

It's now pretty easy to use our similarity matrix to look up the most similar images to a given images. We select a random image:

In [ ]:
query_id = np.random.choice(image_ids)
image_dict[query_id]

and then grab the column in the matrix which holds similarity measures for our `query_id`. We sort the indexes according to their similarity to the query image, and grab the top 5 results (which are not our original query)

In [ ]:
most_similar_ids = naive_similarity[query_id].sort_values().index.values[1:6]

we can now grab the corresponding images and display them

In [ ]:
similar_images = [image_dict[id].resize((300, 300)) for id in most_similar_ids]
Image.fromarray(np.hstack([np.array(image) for image in similar_images]).reshape(300, 1500, 3))

these results are super sketchy... They're not _bad_ exactly, but they're definitely not good. Playing with the results for a while, it's easy to spot where the approach succeeds and where the rough edges appear.

### The problem with histograms
If you've spent any amount of time exploring the collection, you know that _a lot_ of the images exist in a weird, sad, grubby, greyscale/sepia section of the colour space. In that space, the histogram approach is perfectly sufficient (I was actually surprised by how good the results for some of these query images are).  
The method falls down when 
cosine similarity (and almost all other vector similarity metrics) assume that 

To cosine, `red=222` and `red=223` are _entirely_ different colours.

# Smoothed histograms

In [ ]:
def moving_average(arr, n):
    '''
    Returns a moving average over a given array
    
    Parameters
    ----------
    arr : numpy.array
        input array
    n : int
        window size

    Returns
    -------
    arr : numpy.array
        input array with moving average applied
    '''
    cumsum = np.cumsum(arr)
    return (cumsum[n:] - cumsum[:-n])[n - 1:] / n


def smooth_histogram(hist, n=10):
    '''
    applies a moving average to a image histogram, retaining separation between
    the 3 channels
    
    Parameters
    ----------
    hist : list
        flat input histogram of size=(768,)
    n : int
        window size

    Returns
    -------
    arr : numpy.array
        input array with moving average applied
    '''
    r, g, b = np.split(np.array(hist), 3)
    return np.concatenate([moving_average(r, n), 
                           moving_average(g, n), 
                           moving_average(b, n)])

In [ ]:
smooth_histograms = [smooth_histogram(h) for h in histograms]

In [ ]:
smooth_similarity = pd.DataFrame(data=[[cosine(h_1, h_2) 
                                        for h_1 in smooth_histograms] 
                                       for h_2 in tqdm(smooth_histograms)],
                                 index=image_ids,
                                 columns=image_ids)

In [ ]:
sns.heatmap(smooth_similarity);

In [ ]:
query_id = np.random.choice(image_ids)
image_dict[query_id]

In [ ]:
most_similar_ids = smooth_similarity[query_id].sort_values().index.values[1:6]

In [ ]:
similar_images = [image_dict[id].resize((300, 300)) for id in most_similar_ids]
Image.fromarray(np.hstack([np.array(image) for image in similar_images]).reshape(300, 1500, 3))

# Dimensionality reduction
We expect that most neighbours will be correlated with one another, but it's harder to appreciate which neighbourhoods will be more tightly bunched

In [ ]:
histograms = np.array(histograms)
histograms.shape

In [ ]:
reduced_histograms = UMAP(n_components=600).fit_transform(histograms)

In [ ]:
reduced_similarity = pd.DataFrame(data=[[cosine(h_1, h_2) 
                                         for h_1 in reduced_histograms] 
                                        for h_2 in tqdm(reduced_histograms)],
                                  index=image_ids,
                                  columns=image_ids)

sns.heatmap(reduced_similarity);

In [ ]:
query_id = np.random.choice(image_ids)
image_dict[query_id]

In [ ]:
most_similar_ids = reduced_similarity[query_id].sort_values().index.values[1:6]

similar_images = [image_dict[id].resize((300, 300)) for id in most_similar_ids]
Image.fromarray(np.hstack([np.array(image) for image in similar_images]).reshape(300, 1500, 3))

# direct comparison

In [ ]:
query_id = np.random.choice(image_ids)
image_dict[query_id]

In [ ]:
most_similar_ids = naive_similarity[query_id].sort_values().index.values[1:6]
similar_images = [image_dict[id].resize((300, 300)) for id in most_similar_ids]
Image.fromarray(np.hstack([np.array(image) for image in similar_images]).reshape(300, 1500, 3))

In [ ]:
most_similar_ids = smooth_similarity[query_id].sort_values().index.values[1:6]
similar_images = [image_dict[id].resize((300, 300)) for id in most_similar_ids]
Image.fromarray(np.hstack([np.array(image) for image in similar_images]).reshape(300, 1500, 3))